<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Process-TEST-set" data-toc-modified-id="Process-TEST-set-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Process TEST set</a></span></li><li><span><a href="#Process-TRAIN-set" data-toc-modified-id="Process-TRAIN-set-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Process TRAIN set</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import cv2
from skimage import color
from skimage import io

from scipy import misc

In [2]:
TEST = pd.read_csv("Principal: Processed Images/TESTmod.csv", index_col=0)
TRAIN = pd.read_csv("Principal: Processed Images/TRAINmod.csv", index_col=0)

In [3]:
TRAIN.shape


(20432, 4)

In [4]:
TRAIN = TRAIN[TRAIN.origin=="ISIC2018_Task3_Training_Input/"]
TRAIN.sum()

MEL                                                     890
NV                                                     5364
BKL                                                     879
origin    ISIC2018_Task3_Training_Input/ISIC2018_Task3_T...
dtype: object

In [5]:
# Grey world color correction algorithm
#from https://gist.github.com/shunsukeaihara/4603234

def grey_world(nimg):
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    mu_g = np.average(nimg[1])
    nimg[0] = np.minimum(nimg[0]*(mu_g/np.average(nimg[0])),255)
    nimg[2] = np.minimum(nimg[2]*(mu_g/np.average(nimg[2])),255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)

## Process TEST set

In [ ]:
# "Principal: Processed Images/Data v1" function for TEST.csv
# Processes only HAM10000 images
import numpy as np

def process_image(name): # takes filename of the lesion sample
    img = io.imread("ISIC2018_Task3_Training_Input/"+name+".jpg")
    img = grey_world(img)
    img = img[22:22+405, 22:22+405, :]
    img = cv2.resize(img, dsize=(224, 224))#, interpolation=cv2.INTER_CUBIC)

    misc.imsave("Processed Images/TEST/"+ name+".jpg", img)

In [ ]:
import time
from tqdm import tqdm_notebook
start = time.time()
for lesion in tqdm_notebook(TEST.index):
    process_image(lesion)
    
end = time.time()
print("Run time for "+ str(len(TEST.index)) + " images: " + str((end - start)/60)+" minutes")    

In [6]:
from glob import glob
filelist = glob("Processed Images/TEST/ISIC_*.jpg")
len(filelist)

1784

## Process TRAIN set

In [ ]:
# Find out where each image is

MSK1 = glob("ISIC Archival Images/MSK-1/ISIC_*.jpg")
MSK2 = glob("ISIC Archival Images/MSK-2/ISIC_*.jpg")
MSK3 = glob("ISIC Archival Images/MSK-3/ISIC_*.jpg")
MSK4 = glob("ISIC Archival Images/MSK-4/ISIC_*.jpg")
MSK5 = glob("ISIC Archival Images/MSK-5/ISIC_*.jpg")
SONIC = glob("ISIC Archival Images/SONIC/ISIC_*.jpg")
UDA1 = glob("ISIC Archival Images/UDA-1/ISIC_*.jpg")
UDA2 = glob("ISIC Archival Images/UDA-2/ISIC_*.jpg")

allsets = [MSK1, MSK2, MSK3, MSK4, MSK5, SONIC, UDA1, UDA2]
for dataset in allsets:
    for i in range(len(dataset)):
        dataset[i] = dataset[i][27:-4]
HAM = glob("ISIC2018_Task3_Training_Input/ISIC_*.jpg")
for i in range(len(HAM)):
    HAM[i] = HAM[i][30:-4]


In [7]:
def process_steps(img, name): # performs same steps as process_image but saves img in TRAIN
    img = grey_world(img)
    img = cv2.resize(img, dsize=(450, 600))#, interpolation=cv2.INTER_CUBIC)
    xrand = np.random.randint(0,45)
    yrand = np.random.randint(0,45)
    img = img[xrand:xrand+405, yrand:yrand+405, :]
    img = cv2.resize(img, dsize=(256, 256))#, interpolation=cv2.INTER_CUBIC)

    
    misc.imsave("Processed Images/TRAIN/"+ name+".jpg", img)

In [8]:
# "Processed Images/Data" function for TRAIN.csv
# Processes images for each directory

def process_images_all(name):
    img = io.imread(TRAIN.loc[name]["origin"]+name+".jpg")
    process_steps(img, name)
    

In [11]:
import time
from tqdm import tqdm_notebook

start = time.time()
for lesion in tqdm_notebook(TRAIN.index):
    process_images_all(lesion)
     
end = time.time()
print("Run time for "+ str(len(TRAIN.index)) + " images: " + str(round((end - start)/60, 3))+" minutes")    


/Users/alecx/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # Remove the CWD from sys.path while we load stuff.



Run time for 7133 images: 1.963 minutes


In [ ]:
filelist = glob("Principal: Processed Images/TRAIN/ISIC_*.jpg")
len(filelist)